# Inferring the goals of autonomous agents in Gen.jl

Gen.jl is a a runtime system and meta-programming library for compositional probabilistic generative models, inference algorithms, and deep learning. This tutorial will show you how to use Gen.jl to build a probabilistic model of an autonomous agent that has latent beliefs and goals, and use probabilistic inference to infer the agent's goals given their observed behavior. This model is based on research at MIT ProbComp ([Cusumano-Towner et al., 2017](https://arxiv.org/abs/1704.04977)).

In [5]:
using Gen
srand(1);

## 1. Modeling an autonomous agent

In Gen.jl, probabilistic models are represented by probabilistic programs. Probabilistic programs are ```%IRIS: repeated 'are'%``` are programmatic descriptions of stochastic generative processes. A probabilistic program in Gen.jl is very much like a regular Julia function, except that certain expressions ```%IRIS: 'certain expressions' seems ambiguous, maybe elaborate breifly...just an idea%``` in the program are labeled with names using the "`~`" symbol, and that the syntax for defining a probabilistic program is slightly different than the syntax for defining a Julia function. 

Below, we define an `agent_model` probabilistic program that models the beliefs, goals, and goal-directed motion of an autonomous agent. The model assumes the agent starts at a given location in a scene, and desires to move to a goal location. We model the motion of the agent by assuming that the agent walks along a path from the goal ```%IRIS: 'goal' -> 'start'?%``` to its destination that is constructed using a path-planning algorithm based on the rapidly-exploring random tree (RRT) algorithm from robotics. We have implemented the RRT algorithm and simple library for composing scenes in Julia source files.

In [6]:
include("scene.jl")
include("path_planner.jl")
include("uniform_2d.jl");

In [111]:
@program agent_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    
    #%IRIS-MW: remove these lines
#     wall_height = 30.
#     add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
#     add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
#     add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
#     add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
#     add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))   

    #%IRIS-MW: replace with snippet here-------------
    wall_height = 30.
    walls = [
        Wall(Point(20., 40.), 1, 40., 2., wall_height)
        Wall(Point(60., 40.), 2, 40., 2., wall_height)
        Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height)
        Wall(Point(20., 80.), 1, 15., 2., wall_height)
        Wall(Point(20., 40.), 2, 40., 2., wall_height) ] ~ "walls"
    
    for wall in walls
        add!(scene, wall)
    end
    #---------------------------------------
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = uniform_2d(0, 100, 0, 100) ~ "start"
    #start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = uniform_2d(0, 100, 0, 100) ~ "destination"
    #destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene) 
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

We can execute this probabilistic program to generate probable scenarios. When we execute a probabilistic program, we record the values of any named expressions that were encountered during its execution (e.g. `~ "start"`, `~ "destination"`) into a **trace**. We create a new empty trace with `Trace()`, and we execute a probabilistic program and record its expressions into a trace with `@generate(trace, program(args))`:

In [112]:
trace = Trace()
@generate(trace, agent_model())
println("start: ", value(trace, "start"))
println("destination: ", value(trace, "destination"))
println("x1 through x4: ", map((i) -> value(trace, "x$i"), 1:4))
println("y1 through y4: ", map((i) -> value(trace, "y$i"), 1:4))

start: Point(95.54335126843434,85.12600877701198)
destination: Point(41.2303670073407,99.85469286238838)
x1 through x4: [95.1847,87.5117,75.5251,64.8823]
y1 through y4: [85.5091,89.2591,90.8951,93.816]


If we execute the program again it again, we get a different result:

In [113]:
trace = Trace()
@generate(trace, agent_model())
println("start: ", value(trace, "start"))
println("destination: ", value(trace, "destination"))
println("x1 through x4: ", map((i) -> value(trace, "x$i"), 1:4))
println("y1 through y4: ", map((i) -> value(trace, "y$i"), 1:4))

start: Point(62.510476719769684,62.774453126006335)
destination: Point(56.211426383051744,15.069490452820556)
x1 through x4: [62.8811,63.9006,62.9009,60.9558]
y1 through y4: [62.7659,52.2293,42.2132,31.6259]


We can also print the whole trace. For now, don't worry about constraints, interventions, or proposals. ```%IRIS: I'd move the "don't worry about contraints..etc" to the end of this paragraph"%``` Note that the `recorded` section lists the values that each of the named expressions took during the execution:

In [114]:
print(trace)

-- Constraints --
-- Interventions --
-- Proposals --
-- Recorded --
walls => Wall[Wall(Point(20.0,40.0),1,40.0,2.0,30.0,Polygon(Point[Point(20.0,40.0),Point(60.0,40.0),Point(60.0,42.0),Point(20.0,42.0)]),:Wall),Wall(Point(60.0,40.0),2,40.0,2.0,30.0,Polygon(Point[Point(60.0,40.0),Point(62.0,40.0),Point(62.0,80.0),Point(60.0,80.0)]),:Wall),Wall(Point(45.0,80.0),1,17.0,2.0,30.0,Polygon(Point[Point(45.0,80.0),Point(62.0,80.0),Point(62.0,82.0),Point(45.0,82.0)]),:Wall),Wall(Point(20.0,80.0),1,15.0,2.0,30.0,Polygon(Point[Point(20.0,80.0),Point(35.0,80.0),Point(35.0,82.0),Point(20.0,82.0)]),:Wall),Wall(Point(20.0,40.0),2,40.0,2.0,30.0,Polygon(Point[Point(20.0,40.0),Point(22.0,40.0),Point(22.0,80.0),Point(20.0,80.0)]),:Wall)]
times => [0.0,10.5263,21.0526,31.5789,42.1053,52.6316,63.1579,73.6842,84.2105,94.7368,105.263,115.789,126.316,136.842,147.368,157.895,168.421,178.947,189.474,200.0]
start => Point(62.510476719769684,62.774453126006335)
destination => Point(56.211426383051744,15.069490452

# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing a trace is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. In Gen.jl a trace renderer is simply ```%IRIS: add "an"% ``` object that has a method "`render(renderer, trace::Trace)`". In Jupyter notebooks, we render traces using JavaScript code that renders traces onto a Document Object Model (DOM) element in the output of notebook cells.

Below, we define a trace renderer for `agent_model` using JavaScript and CSS. In particular we use the Javascript data visualization library [D3](https://d3js.org/). First we include some CSS to define visual features of the trace renderings:

In [115]:

HTML("""<style>
    circle { 
        r: 2
    }
    .start {
        fill: blue;
    }
    .destination {
        fill: red;
    }
    .waypoint {
        fill: none;
        stroke: black;
        stroke-width: 3;
        stroke-dasharray: 1.5, 1.5;
        
    
        //IRIS:Added waypoint
    } 
    .path {
        fill: orange;
        fill-opacity: 0.3;
    }
    .path_segments {
        stroke: black;
        stroke-opacity: 0.2;
    }
    .wall {
        fill: gray
    }
    .tree {
        fill: green
    }
    .score {
        text-anchor: middle;
        font-size: 10px;
    }
    .interventions {
        stroke: black;
        stroke-width: 1;
    }
    .constraints {
        stroke: black;
        stroke-width: 1;
        stroke-dasharray: 1, 1;
    }
    .legend {
        font-size: 8px;
        alignment-baseline: middle;
    }
</style>""")

HTML{String}("<style>\n    circle { \n        r: 2\n    }\n    .start {\n        fill: blue;\n    }\n    .destination {\n        fill: red;\n    }\n    .waypoint {\n        fill: none;\n        stroke: black;\n        stroke-width: 3;\n        stroke-dasharray: 1.5, 1.5;\n        \n    \n        //IRIS:Added waypoint\n    } \n    .path {\n        fill: orange;\n        fill-opacity: 0.3;\n    }\n    .path_segments {\n        stroke: black;\n        stroke-opacity: 0.2;\n    }\n    .wall {\n        fill: gray\n    }\n    .tree {\n        fill: green\n    }\n    .score {\n        text-anchor: middle;\n        font-size: 10px;\n    }\n    .interventions {\n        stroke: black;\n        stroke-width: 1;\n    }\n    .constraints {\n        stroke: black;\n        stroke-width: 1;\n        stroke-dasharray: 1, 1;\n    }\n    .legend {\n        font-size: 8px;\n        alignment-baseline: middle;\n    }\n</style>")

Next, we define the trace renderer in Javascript, and register it with Gen using the Javascript function `Gen.register_jupyter_renderer`. Each Javascript trace renderer is given a name when it is registered, so that the Julia code can send the trace data to the right renderer. A Javascript trace renderer is a function with signature `function(dom_id, trace, configuration) {.. }`. The first argument `dom_id` defines what element of the DOM the renderer should write to, and `trace` contains the trace data sent from Julia. `%IRIS:maybe don't add this if it's not fixed%` Javascript code in the cell below should have correct Javascript syntax highlighting (you may have to click in the cell to activate the syntax highlighting).

In [116]:
javascript"""

// Gen.jl provides a Jupyter notebook extension that allows trace to be
// passed from Julia to Javscript rendering functions.
var Gen = require("nbextensions/gen_notebook_extension/main");

// We use D3 for the trace rendering in this example notebook. However,
// in principle any Javascript library for graphics or visualization
// can be used to render Gen.jl traces.
var d3 = require("nbextensions/d3/d3.min");

// Add an SVG element representing the scene.
function add_svg(parent, trace) {
    return parent.append("svg")
        .attr("viewBox", "0 0 100 100")
        .attr("position", "absolute")
        .style("height", "100%");
}

function add_svg_if_not_exists(parent, trace) {
    var svg = parent.selectAll("svg").data([""]);
    return svg.enter().append("svg")
        .attr("viewBox", "0 0 100 100")
        .attr("position", "absolute")
        .style("height", "100%")
        .merge(svg);
}

function add_bounding_box(svg) {
    svg.selectAll("rect").data([""]).enter().append("rect")
        .attr("width", "100%")
        .attr("height", "100%")
        .attr("stroke", "#000")
        .attr("fill", "#FFF")
        .attr("fill-opacity", 0.0);
}

// Add the static elements of the scene
function add_scene(svg, trace) {
    var scene = Gen.find_choice(trace, "scene");
    
    var trace_trees = scene.value.obstacles.filter(function(element) { return element.name == "Tree";});
    var trees = svg.selectAll(".tree").data(trace_trees);
    trees.exit().remove();
    trees.enter().append("rect").classed("tree", true).classed("trace", true)
      .merge(trees)
        .attr("x", function(d) { return d.center.x - d.size/2.0; })
        .attr("y", function(d) { return d.center.y - d.size/2.0; })
        .attr("width", function(d) { return d.size; })
        .attr("height", function(d) { return d.size; });
    
    var trace_walls = scene.value.obstacles.filter(function(element) { return element.name == "Wall";});
    var walls = svg.selectAll(".wall").data(trace_walls);
    walls.exit().remove();
    walls.enter().append("rect").classed("wall", true).classed("trace", true)
      .merge(walls)
        .attr("x", function(d) { return d.start.x; })
        .attr("y", function(d) { return d.start.y; })
        .attr("width", function(d) { return d.orientation == 1 ? d.length : d.thickness; })
        .attr("height", function(d) { return d.orientation == 2 ? d.length : d.thickness; });
}

// Add the starting location of the agent
function add_start(svg, trace) {
    var trace_start = Gen.find_choice(trace, "start");
    var start = svg.selectAll(".start").data(trace_start ? [trace_start] : []);
    start.exit().remove();
    start.enter().append("circle").classed("start", true)
      .merge(start)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });
}

// Add the destination location of the agent
function add_destination(svg, trace) {
    var trace_dest = Gen.find_choice(trace, "destination");
    var dest = svg.selectAll(".destination").data(trace_dest ? [trace_dest] : []);
    dest.exit().remove();
    dest.enter().append("circle").classed("destination", true)
      .merge(dest)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });
}
                                                                                                                            
// Add waypoint of a path if given
function add_waypoint(svg, trace) {
    // if exists                                                                                                                                
    var trace_waypoint = Gen.find_choice(trace, "waypoint");
    var waypoint = svg.selectAll(".waypoint").data(trace_waypoint ? [trace_waypoint] : []);
    waypoint.exit().remove();
    waypoint.enter().append("circle").classed("waypoint", true)
      .merge(waypoint)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });
}                                                                                                                            

// Add the path that the agent takes from its start to destination
// Show the measured location of the agent along its path at each 
// of the measured time points.
function add_path(svg, trace, update) {
    var times = Gen.find_choice(trace, "times").value;
    var path_point_data = [];
    for (var i=1; i<=times.length; i++) {
        var x = Gen.find_choice(trace, "x" + i);
        var y = Gen.find_choice(trace, "y" + i);
        if (x && y) {
            path_point_data.push({x: x, y: y, where: x.where == y.where ? x.where : "mixed" });
        }
    }
    var path_segment_data = [];
    for (var i=1; i<times.length; i++) {
        var x_cur = Gen.find_choice(trace, "x" + i);
        var y_cur = Gen.find_choice(trace, "y" + i);
        var x_next = Gen.find_choice(trace, "x" + (i+1));
        var y_next = Gen.find_choice(trace, "y" + (i+1));
        if (x_cur && y_cur && x_next && y_next) {
            path_segment_data.push({prev: {x: x_cur, y: y_cur},
                                    next: {x: x_next, y: y_next}});
        }
    }

    var path_segments = svg.selectAll(".path_segments").data(path_segment_data);
    path_segments.exit().remove();
    path_segments.enter().append("line").classed("path_segments", true).classed("trace", true)
        .merge(path_segments)
        .attr("x1", function(d) { return d.prev.x.value; })
        .attr("y1", function(d) { return d.prev.y.value; })
        .attr("x2", function(d) { return d.next.x.value; })
        .attr("y2", function(d) { return d.next.y.value; });
    
    var path_points = svg.selectAll(".path").data(path_point_data);
    path_points.exit().remove();
    path_points.enter().append("circle").classed("path", true).classed("trace", true)
        .on("mouseover", handleMouseOver)
        .on("mouseout", handleMouseOut)
        .merge(path_points)
        .attr("cx", function(d) { return d.x.value; })
        .attr("cy", function(d) { return d.y.value; });
    
    function handleMouseOver(d, i) {
        var x = d.x.value;
        var y = d.y.value;
        svg.append("text").attr("id", "t" + "-" + i).attr("x", x).attr("y", y)
            .attr("font-size", "6px").attr("pointer-events", "none")
            .text([x.toFixed(1), y.toFixed(1)]);
    };
                              
    function handleMouseOut(d, i) {
        var x = d.x.value;
        var y = d.y.value;
        d3.select("#t" + "-" + i).remove();
    };
}

// Set a different style to elements if they were intervened in the trace
// with intervene! or constrained with constrain!
function apply_styles(svg) {
    svg.selectAll(".trace")
        .classed("interventions", function(d) { return d.where == Gen.interventions; })
        .classed("constraints", function(d) { return d.where == Gen.constraints; });
}

// Add the score of the trace to the bottom of the scene
function add_score(svg, trace) {
    var score = svg.selectAll(".score").data([""]);
    score.enter().append("text").classed("score", true)
        .attr("x", 50).attr("y", 95)
        .text(trace.log_weight.toFixed(2));
}

// The main Javascript trace rendering function, registered with Gen
// Renders 'trace' onto the the DOM element #'id'
Gen.register_jupyter_renderer("agent_model_renderer", function(id, trace, conf) {
    var root = d3.select("#" + id);
    var svg;
    switch (conf.mode) {
        case "overlay":
            root = add_svg_if_not_exists(root, trace);
            svg = add_svg(root, trace);
            break;
        case "overwrite":
            svg = add_svg_if_not_exists(root, trace);
            break;
        default:
            break;
    }
    add_bounding_box(svg);
    if (conf.show_path) {
        add_path(svg, trace, false);
    }
    add_start(svg, trace);
    add_destination(svg, trace);
    apply_styles(svg);
    add_scene(svg, trace);
    add_waypoint(svg, trace);
    if (conf.show_score) {
        add_score(svg, trace);
    }
});

// An secondary trace rendering that just draws a legend onto the DOM element #'id'
Gen.register_jupyter_renderer("agent_model_legend_renderer", function(id, trace, conf) {
    var root = d3.select("#" + id);
    root.append("circle").classed("start", true).attr("cx", 10).attr("cy", 10);
    root.append("text").text("Starting location").classed("legend", true).attr("x", 20).attr("y", 10);
    root.append("circle").classed("destination", true).attr("cx", 10).attr("cy", 20);
    root.append("text").text("Destination").classed("legend", true).attr("x", 20).attr("y", 20);
    root.append("circle").classed("path", true).attr("cx", 10).attr("cy", 30);
    root.append("text").text("Location on path").classed("legend", true).attr("x", 20).attr("y", 30);
    
    root.append("rect").classed("tree", true).attr("x", 5).attr("y", 35).attr("width", 10).attr("height", 10);
    root.append("text").text("Tree").classed("legend", true).attr("x", 20).attr("y", 40);
    root.append("rect").classed("wall", true).attr("x", 5).attr("y", 50).attr("width", 10).attr("height", 2);
    root.append("text").text("Wall").classed("legend", true).attr("x", 20).attr("y", 50);
    
    root.append("circle").classed("interventions", true).attr("cx", 10).attr("cy", 60).style("fill", "white");
    root.append("text").text("Intervened").classed("legend", true).attr("x", 20).attr("y", 60);
    root.append("circle").classed("constraints", true).attr("cx", 10).attr("cy", 70).style("fill", "white");
    root.append("text").text("Constrained").classed("legend", true).attr("x", 20).attr("y", 70);
    
    // %IRIS: added next two lines for waypoint in legend%
    root.append("circle").classed("waypoint", true).attr("cx", 10).attr("cy", 80)
    root.append("text").text("Waypoint").classed("legend", true).attr("x", 20).attr("y", 80);
});
"""

We create a DOM element that the trace renderer renders to ```%IRIS: to what?/remove 'to'%``` using the `Figure` function in Julia. The `here(figure)` places a DOM element in the output of the cell onto which Javascript trace renderers will be able to render traces. It should appear as as ```%IRIS: redundant 'as' %``` a mostly empty output area.

In [141]:
#%IRIS: MAYBE WARN THE USER MORE BOLDLY... THAT THIS IS SUPPOSE TO BE EMPTY... 
#people like running code presumptuously... like me%"
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20,
                titles=["Trace"])
here(figure)

HTML{String}("    <div id=\"id_84c954ebf13e4b1ab67fe7b325ce5c92\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_84c954ebf13e4b1ab67fe7b325ce5c92\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 400);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 2;\n        var tile_height = 400 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<2; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_84c954ebf13e4b1ab67fe7b325ce5c92_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_84c954ebf13e4b1ab67fe7b325ce5c92_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_84c954ebf13e4b1ab67fe7b325ce5c92_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"Trace\")\n            .style(\"font-size\", 16);")

Now, we construct a Julia trace renderer object that will delegate rendering to the Javascript trace renderer that we defined above.

In [142]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite", "show_path" => true));

We then execute the program, and render the resulting trace. You should see the trace rendering appear on the left side of the rendering area above.

In [163]:
trace = Trace()
@generate(trace, agent_model())
attach(renderer, figure => 1)
render(renderer, trace)

We also wrote a renderer that just shows the legend. We ask that renderer to put a legend in the right side of the rendering area above.

In [140]:

legend_renderer = JupyterInlineRenderer("agent_model_legend_renderer", Dict())
attach(legend_renderer, figure => 2)
render(legend_renderer, trace)

A probabilistic program is stochastic, and the distribution over its executions defines a probability distribution over potential scenarios. Let's get a sense of this probability distribution by rendering many executions, one after another, in an animation:

In [121]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20,
                titles=["Trace"])
here(figure)

HTML{String}("    <div id=\"id_23cd413d7cbb4609bccdbe0a44621d11\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_23cd413d7cbb4609bccdbe0a44621d11\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 400);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 2;\n        var tile_height = 400 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<2; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_23cd413d7cbb4609bccdbe0a44621d11_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_23cd413d7cbb4609bccdbe0a44621d11_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_23cd413d7cbb4609bccdbe0a44621d11_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"Trace\")\n            .style(\"font-size\", 16);")

In [122]:
attach(legend_renderer, figure => 2)
render(legend_renderer, trace)
attach(renderer, figure)
for i=1:100
    trace = Trace()
    @generate(trace, agent_model())
    render(renderer, trace)
end

We can also render many runs side by side in a grid, instead of an animation. This also gives a sense of the probability distribution represented by the probabilistic program.

In [123]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_52d3cad81cb3457cb3e21d8aef6ea1df\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_52d3cad81cb3457cb3e21d8aef6ea1df\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 300 / 2;\n        var i = 1;\n        for (var row=0; row<2; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_52d3cad81cb3457cb3e21d8aef6ea1df_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_52d3cad81cb3457cb3e21d8aef6ea1df_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [124]:
for i=1:12
    trace = Trace()
    @generate(trace, agent_model())
    attach(renderer, (figure => i))
    render(renderer, trace)
end

A key feature of Gen.jl is the ability to constrain the values of random choices that a probabilistic program makes during the course of its execution. Let's see how we can modify the behavior of the program if we constrain the starting location of the agent and its destination location using the `constrain!(trace, name, value)` method. We will constrain the start location to lie in the upper left corner and the destination to lie in the bottom right corner.

In [125]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_bcf96fc18b7f41daa7c4be4cacf82cea\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_bcf96fc18b7f41daa7c4be4cacf82cea\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 300 / 2;\n        var i = 1;\n        for (var row=0; row<2; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_bcf96fc18b7f41daa7c4be4cacf82cea_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_bcf96fc18b7f41daa7c4be4cacf82cea_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [126]:
for i=1:12
    trace = Trace()
    constrain!(trace, "start", Point(10, 10))
    constrain!(trace, "destination", Point(90, 90))
    @generate(trace, agent_model())
    attach(renderer, (figure => i))
    render(renderer, trace)
end

Now we have a sense of the variability in the paths that the agent takes.

We can also get a sense for the variability by overlaying many traces on top of one another:

In [127]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20,
                titles=["Trace"])
here(figure)

HTML{String}("    <div id=\"id_f729e3a9cceb47c98aab9fd0c9bcd29b\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_f729e3a9cceb47c98aab9fd0c9bcd29b\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 400);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 2;\n        var tile_height = 400 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<2; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_f729e3a9cceb47c98aab9fd0c9bcd29b_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_f729e3a9cceb47c98aab9fd0c9bcd29b_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_f729e3a9cceb47c98aab9fd0c9bcd29b_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"Trace\")\n            .style(\"font-size\", 16);")

In [128]:
attach(legend_renderer, figure => 2)
render(legend_renderer, trace)
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay", "show_path" => true))
attach(renderer, figure)
for i=1:20
    trace = Trace()
    intervene!(trace, "start", Point(10, 10))
    intervene!(trace, "destination", Point(90, 90))
    @generate(trace, agent_model())
    render(renderer, trace)
end

# 3. Probabilistic inference

So far, we have generated scenarios that the model thinks are probable, and we have constrained some random choices and simulated the consequence. However, suppose we had observed the start location of the agent, and a certain sequence of locations of the agent along its path, and we wanted to know probable goal locations?

This is a query that cannot be answered simply by forward simulation of the program, because the location of the agent is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes as we did above, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 7 time points:

In [129]:
#%IRIS: I think it would be really nice to show a plot of the locations - instead of just showing the point objects%
dataset = [
    Point(10.3867,10.3889)
    Point(11.0188,20.0843)
    Point(11.6663,30.0142)
    Point(11.4507,41.5159)
    Point(13.8081,53.0258)
    Point(13.1958,61.3572)
    Point(17.1566,73.7131)
]

7-element Array{Point,1}:
 Point(10.3867,10.3889)
 Point(11.0188,20.0843)
 Point(11.6663,30.0142)
 Point(11.4507,41.5159)
 Point(13.8081,53.0258)
 Point(13.1958,61.3572)
 Point(17.1566,73.7131)

The first step in inference is to constrain the random choices that are observed.

In [130]:
trace = Trace()
constrain!(trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

Now, when we run the program in this trace, we find that the score of the trace tells us how well the other values in the trace match the value of the constraints. `%IRIS weird sentence -> %`Very negative scores that the other values in the trace, such as the destination and the projected path, do not match well with the constrained path points. The path points that were constrained are identified in the trace renderings by a dashed border.

In [131]:
figure = Figure(num_rows=3, num_cols=3, width=900, height=900, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_d08d4b511d244d48a9ce303ffc317aad\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_d08d4b511d244d48a9ce303ffc317aad\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 900);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 900 / 3;\n        var i = 1;\n        for (var row=0; row<3; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_d08d4b511d244d48a9ce303ffc317aad_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_d08d4b511d244d48a9ce303ffc317aad_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [132]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay",
                                                              "show_path" => true,
                                                              "show_score" => true))
for i=1:30
    t = deepcopy(trace)
    @generate(t, agent_model())
    attach(renderer, (figure => i))
    render(renderer, t)
end

Given a large number of traces generated by the program, and a score for each trace that tells us how well that trace matches the constraints, `%IRIS: rest of this sentence sounds odd%` we can select those trace that match the constraints. These traces then represent plausible hypotheses that can explain the observed data. This is the approach taken by the two basic inference algorithms that we implement next, *Importance Sampling* and *Metropolis-Hastings*.

### Importance sampling

The code cell below shows a basic importance sampling procedure that uses the score of traces to probabilistically filter them and tries to select a trace that explains the data well. Note that this procedure is a regular Julia function.

In [164]:
function agent_model_importance_sampling(trace::Trace, num_samples::Int)
    
    # the 'trace' argument contains the constraints that represent the 
    # observed dataset. 
    
    # perform many independent executions of the probabilistic program, and 
    # record each execution in a trace. store the 'score' for each trace, 
    # which encodes how well the trace matches the constraints
    traces = Vector{Trace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        @generate(t, agent_model())
        scores[k] = score(t)
        traces[k] = t
    end
    
    # compute a %IRIS:misspelled ->% reltaive weight for each trace indicating how well the trace
    # matches the constraints relative to the other traces generated. A large
    # weight indicates that the trace matches the constraints well, and may be
    # a better explanation for the observed data than other %IRIS:'traces'->% trace.
    
    # weights = exp(scores) / sum(exp(scores))
    weights = exp(scores - logsumexp(scores))
    weights = weights / sum(weights)
    
    # pick a trace in propotion to its relative weight and return it
    chosen = rand(Categorical(weights))
    return traces[chosen]
end;

Because this Julia function treats the execution traces of the probabilistic program `agent_model` as data, this inference procedure is an example of a [meta-program](https://en.wikipedia.org/wiki/Metaprogramming). In Gen.jl, inference is done using meta-programs that reason about the traces of other programs. This is in contrast to most other probabilistic programming systems, in which users do not write their own custom meta-programs, but instead rely on built-in generic inference techniques.

Let's see the output of our importance sampling procedure. Recall that each run of the procedure produces one trace of the `agent_model` probabilistic program. Note that the importance sampling procedure is stochastic---repeated executions will generate different traces. This is an important feature---we want inference algorithms to give us probable explanations for the data, but also a sense of how uncertain we are in those explanations. By executing the inference algorithm many times, we can get a sense of the uncertainty in the inferences.

In [165]:
all_num_simulations = [1, 4, 16]  #%IRIS: bug <- Originally 'num_samples_list'. Should be 'all_num_simulations'
figure = Figure(num_rows=1, num_cols=length(all_num_simulations),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", all_num_simulations))
here(figure)

HTML{String}("    <div id=\"id_24f5b8aac961433ba71976c4152af685\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_24f5b8aac961433ba71976c4152af685\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_24f5b8aac961433ba71976c4152af685_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_24f5b8aac961433ba71976c4152af685_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_24f5b8aac961433ba71976c4152af685_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_24f5b8aac961433ba71976c4152af685_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_24f5b8aac961433ba71976c4152af685_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 samples)\")\n            .style(\"font-size\", 16);")

In [166]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay", "show_path" => true,
                                                              "show_score" => false))
num_approximate_samples = 30
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
for (i, num_samples) in enumerate(all_num_simulations)
    attach(renderer, (figure => i))
    title =  "SIR ($num_samples particles)"
    for j=1:num_approximate_samples
        output_trace = agent_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

We ran the inference algorithm using `num_samples=1`, `num_samples=4`, and `num_samples=16`. Note that when we increase `num_samples` we get more plausible traces as output. For `num_samples=16`, the algorithm tells us that it thinks the agent is headed either into the box or in the bottom left or bottom of the scene. Also note that the more plausible answers come at a greater computation cost---we have to wait longer for each new sample to arrive. Understanding the time-accuracy tradeoffs in approximate probabilistic inference algorithms is an active area of research (see [Cusumano-Towner, Mansinghka 2016](https://arxiv.org/abs/1705.07224) for recent work in this area).



`%IRIS-MW adding a section on adding a gap`

Next, we change the scene of the generative agent model by adding an addition gap to the walls toward the top of the map. We do this by intervening the "walls" variable in the model. 

In [169]:
#`%IRIS-MW adding Figures`

all_num_simulations = [1, 4, 16]  
figure = Figure(num_rows=1, num_cols=length(all_num_simulations),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", all_num_simulations))
here(figure)

HTML{String}("    <div id=\"id_f08576098b0f4776bd9e37cd9d948150\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_f08576098b0f4776bd9e37cd9d948150\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_f08576098b0f4776bd9e37cd9d948150_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_f08576098b0f4776bd9e37cd9d948150_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_f08576098b0f4776bd9e37cd9d948150_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_f08576098b0f4776bd9e37cd9d948150_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_f08576098b0f4776bd9e37cd9d948150_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 samples)\")\n            .style(\"font-size\", 16);")

In [170]:
#`%IRIS-MW adding SIR and intervening walls`

renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay", "show_path" => true,
                                                              "show_score" => false))
num_approximate_samples = 30
trace = Trace()

# Change the scene of the generative model by adding an additional gap to the walls
wall_height = 30.
new_walls = [
    Wall(Point(60.-15., 40.), 1, 15. + 2., 2., wall_height)
    Wall(Point(20., 40.), 1, 15., 2., wall_height)

    Wall(Point(60., 40.), 2, 40., 2., wall_height)
    Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height)
    Wall(Point(20., 80.), 1, 15., 2., wall_height)
    Wall(Point(20., 40.), 2, 40., 2., wall_height) 
    ]

intervene!(trace, "walls", new_walls)
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
for (i, num_samples) in enumerate(all_num_simulations)
    attach(renderer, (figure => i))
    title =  "SIR ($num_samples particles)"
    for j=1:num_approximate_samples
        output_trace = agent_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

`%IRIS-MW adding observations about the results of SIR`

Notice how the inferences change about the most likely paths for the constrained points. This time, inference demostrates the path is more unlikely to go into the box area. This is due to the gap added to the walls. Assuming the agent is rational (by taking shortest paths to its goal locations), the path would have started showing earlier intentions of entering into the area in between the walls. Therefore, inference shows the agent's goal location is most likely outside the box area. 


### Metropolis-Hastings Inference

Here, we show a basic Metropolis-Hastings inference allgorithm written in Gen.jl that operates according to similar principles. Instead of generating a bunch of traces and then scoring each and selecting it as output, Metropolis-Hastings generates traces one by one and decides to stochastically replace the previous trace with a new trace if the new trace has a higher score than the previous trace.

In [66]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite",
                                                              "show_path" => true,
                                                              "show_score" => true))
figure = Figure(num_rows=1, num_cols=3, width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=["proposed trace", "current trace"])
here(figure)

HTML{String}("    <div id=\"id_4b787377112f43579c042c9d6ec2a408\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_4b787377112f43579c042c9d6ec2a408\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_4b787377112f43579c042c9d6ec2a408_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_4b787377112f43579c042c9d6ec2a408_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_4b787377112f43579c042c9d6ec2a408_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"proposed trace\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_4b787377112f43579c042c9d6ec2a408_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"current trace\")\n            .style(\"font-size\", 16);")

In [67]:
attach(legend_renderer, figure => 3)
render(legend_renderer, trace)
current_trace = Trace()
intervene!(current_trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset) #%IRIS: BUG <- 'points' should be 'dataset'
    constrain!(current_trace, "x$i", point.x)
    constrain!(current_trace, "y$i", point.y)
end
@generate(current_trace, agent_model())
current_score = score(current_trace)
for i=1:100
    proposed_trace = deepcopy(current_trace)
    @generate(proposed_trace, agent_model())
    proposed_score = score(proposed_trace)
    if log(rand()) < proposed_score - current_score
        current_trace = proposed_trace
        current_score = proposed_score
    end
    attach(renderer, figure => 1)
    render(renderer, proposed_trace)
    attach(renderer, figure => 2)
    render(renderer, current_trace)
    sleep(0.1)
end

# 4. Improving the model

The probabilistic model `agent_model` used above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that has a detour in it, and that does not match our model's expectations.

In [68]:
#%IRIS: probably should be combined with the code box below
detour_dataset = [
    Point(9.59825,8.92063)
    Point(21.8936,9.54817)
    Point(30.9534,10.8819)
    Point(43.1137,9.75395)
    Point(48.8929,10.4189)
    Point(46.0282,21.7662)
    Point(35.0281,25.9994)
    Point(27.2084,33.5729)
    Point(20.1662,39.9398)
    Point(18.7309,50.0026)
];

In [69]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20)
here(figure)

HTML{String}("    <div id=\"id_c67471c7e5874afe8758bbd7b1aaeebe\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_c67471c7e5874afe8758bbd7b1aaeebe\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 400);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 2;\n        var tile_height = 400 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<2; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_c67471c7e5874afe8758bbd7b1aaeebe_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_c67471c7e5874afe8758bbd7b1aaeebe_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [70]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite",
                                                              "show_path" => true,
                                                              "show_score" => false))
attach(renderer, figure => 1)

# show the dataset above.
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
@generate(trace, agent_model())

# just show the observed data (don't show the predictions for the remainder of the path)
# remove these from the trace to prevent them from being rendered
delete!(trace, "destination")
for i=length(detour_dataset)+1:20
    delete!(trace, "x$i")
    delete!(trace, "y$i")
end
render(renderer, trace)

# show a legend
attach(legend_renderer, figure => 2)
render(legend_renderer, trace)

Let's see what happens when we try to do probabilistic inference given this dataset, using the `agent_model` model. We will use the same importance sampling algorithm we wrote above, with the same settings for the  `num_samples` parameter.

In [71]:
num_samples_list = [1, 4, 16]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", num_samples_list))
here(figure)

HTML{String}("    <div id=\"id_b24fce6f85dc4a118fc410ee23909c27\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_b24fce6f85dc4a118fc410ee23909c27\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_b24fce6f85dc4a118fc410ee23909c27_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_b24fce6f85dc4a118fc410ee23909c27_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_b24fce6f85dc4a118fc410ee23909c27_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_b24fce6f85dc4a118fc410ee23909c27_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_b24fce6f85dc4a118fc410ee23909c27_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 samples)\")\n            .style(\"font-size\", 16);")

In [72]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay",
                                                              "show_path" => true,
                                                              "show_score" => false))
num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, figure => i)
    for j=1:num_approximate_samples
        output_trace = agent_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

The inferences do not look intuitive. This is because the model `agent_model` cannot explain the detour. It assumes that the detour must be the goal, an `%IRIS: 'an' should be 'and%` it explains the observed data as a very unlikely accident of noise. This is an  example of **model mis-specification**.  In order to make reasonablee inferences for datasets that may contain a detour, we need to improve our model. Here is a new probabilistic program that models an agent that may or may not use an arbitrary waypoint (e.g. detour) when planning its path from its starting location to its destination:

In [73]:
@everywhere @program agent_waypoint_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
    add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
    add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
    add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
    add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))    
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = uniform_2d(0, 100, 0, 100) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = uniform_2d(0, 100, 0, 100) ~ "destination"
    
    if (flip(0.5) ~ "use-waypoint")
        waypoint = uniform_2d(0, 100, 0, 100) ~ "waypoint"
        (tree1, rough_path1, final_path1) = plan_path(start, waypoint, scene)
        (tree2, rough_path2, final_path2) = plan_path(waypoint, destination, scene)
        
        # if either path planner sub-problem failed, then no path was found (final_path is null)
        if isnull(final_path1) || isnull(final_path2)
            final_path = Nullable{Path}() # null
        else
            final_path = Nullable{Path}(concatenate(get(final_path1), get(final_path2)))
        end
    else
        (tree, rough_path, final_path) = plan_path(start, destination, scene)
    end
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed %IRIS: mispellings here
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

Here are some simulations from the program, for a constrained start and destination that both lie one side of the scene:

In [94]:
figure = Figure(num_rows=3, num_cols=6, width=900, height=450, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_048e99b06613406e95e5467d1a888d9d\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_048e99b06613406e95e5467d1a888d9d\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 450);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 450 / 3;\n        var i = 1;\n        for (var row=0; row<3; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_048e99b06613406e95e5467d1a888d9d_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_048e99b06613406e95e5467d1a888d9d_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [95]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite", "show_path" => true))
traces = []
for i=1:18
    trace = Trace()
    constrain!(trace, "start", Point(10, 10))
    constrain!(trace, "destination", Point(30, 90))
    @generate(trace, agent_waypoint_model())
    attach(renderer, figure => i)
    render(renderer, trace)
    push!(traces, trace)
end

Notice that sometimes, the path has a clear waypoint/detour whereas other times it does not. Run the above cell a few times if you don't initially see a clear waypoint. The current trace rendering does not show the waypoint explicitly, but it could be extended to do so.

Let's do inference in this new model on the detour dataset. We'll use importance sampling again. The only change in this procedure from the earlier importance sampling procedure is the choice of model (`agent_waypoint_model` instead of `agent_model`).

In [90]:
@everywhere function agent_waypoint_model_importance_sampling(trace::Trace, num_samples::Int)
    traces = Vector{Trace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        @generate(t, agent_waypoint_model())
        scores[k] = score(t)
        traces[k] = t
    end
    weights = exp(scores - logsumexp(scores))
    weights = weights / sum(weights)
    chosen = rand(Categorical(weights))
    return traces[chosen]
end

In [91]:
num_samples_list = [1, 4, 16]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", num_samples_list))
here(figure)

HTML{String}("    <div id=\"id_35dc4809a1614fb99409ec3053a3d7ef\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_35dc4809a1614fb99409ec3053a3d7ef\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_35dc4809a1614fb99409ec3053a3d7ef_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_35dc4809a1614fb99409ec3053a3d7ef_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_35dc4809a1614fb99409ec3053a3d7ef_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_35dc4809a1614fb99409ec3053a3d7ef_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_35dc4809a1614fb99409ec3053a3d7ef_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 samples)\")\n            .style(\"font-size\", 16);")

In [92]:
trace = Trace()
constrain!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay", "show_path" => true, "show_score" => false))
num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, figure => i)
    for j=1:num_approximate_samples
        output_trace = agent_waypoint_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

It looks like the results are not accurate. Let's try more computation. This may take a few minutes to run. `%IRIS probably put a warning that it's about 10-15 min%`

In [93]:
# TODO parallelize this
num_samples_list = [64, 256, 1024]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n)-> "SIR ($n samples)", num_samples_list))
here(figure)

HTML{String}("    <div id=\"id_e086dfac77744dfeb33c2a76feb04baf\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_e086dfac77744dfeb33c2a76feb04baf\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_e086dfac77744dfeb33c2a76feb04baf_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_e086dfac77744dfeb33c2a76feb04baf_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_e086dfac77744dfeb33c2a76feb04baf_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (64 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_e086dfac77744dfeb33c2a76feb04baf_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (256 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_e086dfac77744dfeb33c2a76feb04baf_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1024 samples)\")\n            .style(\"font-size\", 16);")

In [94]:
num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, figure => i)
    for j=1:num_approximate_samples
        output_sample = agent_waypoint_model_importance_sampling(trace, num_samples)
        render(renderer, output_sample)
    end
end

LoadError: InterruptException:

The extra computation seems to have given us reasonable inferences. Recall that inference in the original model with a typical dataset only required 16 particles to give reasonalbe results for a particular dataset. Why do we need to do more computation to get accurate inferences for `agent_waypoint_path` than we needed for `agent_model`? This is because a random execution of the improved model is a lot less likely to match a typical dataset generated from it than a random execution of the original model was to match a typical dataset generated from it. Conceptually, the distribution on datasets produced by `agent_waypoint_path` has more entropy than the distribution on datasets produced by `agent_path`. This means we need to increase the number of samples to incrase `%IRIS:'increase'` the probability that we get one that matches the data.

# 5. Compiling inference with neural networks

The inference algorithms we gave above don't seem very intelligent---they basically boil down to randomless guessing scenarios and hoping that one happens to match the dataset. There are a number of approaches for creating more efficient inference algorithms. We will focus on one approach based on using neural networks to speed up inference.

First, let's understand in a bit more detail why the importance sampling inference algorithm was slow when applied to the new model with the waypoint.
Suppose we knew the right waypoint:

In [96]:
trace = Trace()
constrain!(trace, "use-waypoint", true)
constrain!(trace, "waypoint", Point(50, 10));

Then the baseline importance sampling algorithm gives reasonable inferences with fewer samples:

In [97]:
num_samples_list = [1, 4, 32]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n)-> "SIR ($n samples)", num_samples_list))
here(figure)

HTML{String}("    <div id=\"id_93fe81dce1a74bb5a92e6246e03fe4cc\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_93fe81dce1a74bb5a92e6246e03fe4cc\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 3;\n        var tile_height = 300 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<3; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_93fe81dce1a74bb5a92e6246e03fe4cc_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_93fe81dce1a74bb5a92e6246e03fe4cc_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_93fe81dce1a74bb5a92e6246e03fe4cc_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_93fe81dce1a74bb5a92e6246e03fe4cc_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_93fe81dce1a74bb5a92e6246e03fe4cc_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (32 samples)\")\n            .style(\"font-size\", 16);")

In [98]:
constrain!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, figure => i)
    for j=1:num_approximate_samples
        output_sample = agent_waypoint_model_importance_sampling(trace, num_samples)
        render(renderer, output_sample)
    end
end

 We use this idea by training a neural network to make informed guesses about the waypoint, given the observed data as its input. We train the neural network on many independent unconstrained executions of the program, each of which generates a trace containing both latent variables and a dataset. The neural network will be trained to predict the waypoint given the dataset.

In [ ]:
# show the neural network, and show the training.

Let's visualize the guesses made by the neural network for a few  different datasets:

In [ ]:
# show four renderings left to right of different datasets, with circles denoting the neural network's guess about the waypoint. 

Now, let's use this trained neural network to speed up inference:

In [ ]:
# show the modififed SIR algorithm, using propose!

In [ ]:
# show results for fewer particles, which should be noticeably faster than without the neural network.
# make an explicit comparison.